In [ ]:

# Chapter 4: Visualize
# A Hands-On Guide to Biomechanics Data Analysis with Python and AI

# This notebook demonstrates visualization of motion capture data using C3D files.
# Author: Dr. Hossein Mokhtarzadeh

# Colab link placeholder (replace with your repo link when uploaded)
# Open in Colab: https://colab.research.google.com/github/hmok/BiomechPythonAI_Guide/blob/main/notebooks/Chapter04_Visualize_WORKING.ipynb

# --- Setup ---
import os, urllib.request, zipfile
import numpy as np
import matplotlib.pyplot as plt

# Download sample data
url = "https://www.c3d.org/data/Sample00.zip"
zip_path = "Sample00.zip"
urllib.request.urlretrieve(url, zip_path)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("c3d_samples")

# Install ezc3d
!pip install ezc3d

import ezc3d

# Find a C3D file
c3d_files = []
for root, dirs, files in os.walk("c3d_samples"):
    for f in files:
        if f.lower().endswith('.c3d'):
            c3d_files.append(os.path.join(root, f))

if not c3d_files:
    raise FileNotFoundError("No C3D files found.")

c3d_path = c3d_files[0]
c3d = ezc3d.c3d(c3d_path)
print("Loaded:", c3d_path)

# --- Plot marker trajectories ---
points = c3d['data']['points']
point_labels = c3d['parameters']['POINT']['LABELS']['value']
n_frames = points.shape[2]

# Choose first marker
marker_idx = 0
marker = points[:3, marker_idx, :].T

plt.figure(figsize=(10,4))
plt.plot(marker[:,0], label='X')
plt.plot(marker[:,1], label='Y')
plt.plot(marker[:,2], label='Z')
plt.title(f"Marker trajectory: {point_labels[marker_idx]}")
plt.xlabel("Frame")
plt.ylabel("Position (mm)")
plt.legend()
plt.grid(True)
plt.show()

# --- 3D path ---
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(marker[:,0], marker[:,1], marker[:,2])
ax.set_title(f"3D Path: {point_labels[marker_idx]}")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()

# --- Plot first analog channel (force plate, EMG, etc.) ---
if 'ANALOG' in c3d['parameters']:
    analog_labels = c3d['parameters']['ANALOG']['LABELS']['value']
    analog_rate = c3d['parameters']['ANALOG']['RATE']['value'][0]
    analogs = c3d['data']['analogs']

    # Use first channel
    signal = analogs[0,0,:]
    t = np.arange(signal.shape[0]) / analog_rate

    plt.figure(figsize=(10,4))
    plt.plot(t, signal)
    plt.title(f"Analog signal: {analog_labels[0]}")
    plt.xlabel("Time (s)")
    plt.ylabel("Signal")
    plt.grid(True)
    plt.show()
else:
    print("No analog channels found.")

# --- Normalized cycle plot ---
# Example: use threshold on analog to detect cycles
if 'ANALOG' in c3d['parameters']:
    thresh = 0.05 * np.max(signal)
    events = np.where(signal > thresh)[0]
    if len(events) > 0:
        stride_len = events[-1] - events[0]
        stride = signal[events[0]:events[-1]]
        stride_norm = np.interp(np.linspace(0,100,101), np.linspace(0,100,len(stride)), stride)

        plt.figure(figsize=(10,4))
        plt.plot(np.linspace(0,100,101), stride_norm)
        plt.title("Normalized Cycle Signal")
        plt.xlabel("Gait Cycle (%)")
        plt.ylabel("Signal")
        plt.grid(True)
        plt.show()
